In [4]:
import pandas as pd
import numpy as np

college = pd.read_csv(r'college-salaries/salaries-by-college-type.csv')

# NOTEBOOK CREATED BY: Vincent Lao
# https://www.kaggle.com/wsj/college-salaries

In [5]:
college.head(5)

,School Name,School Type,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
0,Massachusetts Institute of Technology (MIT),Engineering,"$72,200.00","$126,000.00","$76,800.00","$99,200.00","$168,000.00","$220,000.00"
1,California Institute of Technology (CIT),Engineering,"$75,500.00","$123,000.00",NaN,"$104,000.00","$161,000.00",NaN
2,Harvey Mudd College,Engineering,"$71,800.00","$122,000.00",NaN,"$96,000.00","$180,000.00",NaN
3,"Polytechnic University of New York, Brooklyn",Engineering,"$62,400.00","$114,000.00","$66,800.00","$94,300.00","$143,000.00","$190,000.00"
4,Cooper Union,Engineering,"$62,200.00","$114,000.00",NaN,"$80,200.00","$142,000.00",NaN


In [13]:
college.isnull().sum()

School Name                           0
School Type                           0
Starting Median Salary                0
Mid-Career Median Salary              0
Mid-Career 10th Percentile Salary    38
Mid-Career 25th Percentile Salary     0
Mid-Career 75th Percentile Salary     0
Mid-Career 90th Percentile Salary    38
dtype: int64

In [14]:
college.shape

(269, 8)

In [19]:
region = pd.read_csv(r'college-salaries/salaries-by-region.csv')
region.head()

,School Name,Region,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
0,Stanford University,California,"$70,400.00","$129,000.00","$68,400.00","$93,100.00","$184,000.00","$257,000.00"
1,California Institute of Technology (CIT),California,"$75,500.00","$123,000.00",NaN,"$104,000.00","$161,000.00",NaN
2,Harvey Mudd College,California,"$71,800.00","$122,000.00",NaN,"$96,000.00","$180,000.00",NaN
3,"University of California, Berkeley",California,"$59,900.00","$112,000.00","$59,500.00","$81,000.00","$149,000.00","$201,000.00"
4,Occidental College,California,"$51,900.00","$105,000.00",NaN,"$54,800.00","$157,000.00",NaN


In [20]:
region.isnull().sum()

School Name                           0
Region                                0
Starting Median Salary                0
Mid-Career Median Salary              0
Mid-Career 10th Percentile Salary    47
Mid-Career 25th Percentile Salary     0
Mid-Career 75th Percentile Salary     0
Mid-Career 90th Percentile Salary    47
dtype: int64

In [21]:
region.shape

(320, 8)

In [22]:
region["Mid-Career 10th Percentile Salary"].head()

0    $68,400.00
1           NaN
2           NaN
3    $59,500.00
4           NaN
Name: Mid-Career 10th Percentile Salary, dtype: object

In [23]:
college[college['School Name'].str.contains('University of California')]
# choose the indices (rows) from college where the 'School Name' contains the string 'University of California'

,School Name,School Type,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
21,"University of California, Santa Barbara (UCSB)",Party,"$50,500.00","$95,000.00","$51,300.00","$71,200.00","$129,000.00","$173,000.00"
94,"University of California, Berkeley",State,"$59,900.00","$112,000.00","$59,500.00","$81,000.00","$149,000.00","$201,000.00"
97,University of California at Los Angeles (UCLA),State,"$52,600.00","$101,000.00","$51,300.00","$72,500.00","$139,000.00","$193,000.00"
98,"University of California, San Diego (UCSD)",State,"$51,100.00","$101,000.00","$51,700.00","$75,400.00","$131,000.00","$177,000.00"
99,"University of California, Davis",State,"$52,300.00","$99,600.00","$52,000.00","$71,600.00","$135,000.00","$202,000.00"
101,"University of California, Irvine (UCI)",State,"$48,300.00","$96,700.00","$47,800.00","$66,000.00","$123,000.00","$172,000.00"
108,"University of California, Santa Barbara (UCSB)",State,"$50,500.00","$95,000.00","$51,300.00","$71,200.00","$129,000.00","$173,000.00"
143,"University of California, Santa Cruz (UCSC)",State,"$44,700.00","$84,100.00","$46,100.00","$62,000.00","$121,000.00","$165,000.00"
163,"University of California, Riverside (UCR)",State,"$46,800.00","$81,300.00","$37,200.00","$59,900.00","$109,000.00","$134,000.00"


In [24]:
cols = ["School Name", "Starting Median Salary", "Mid-Career Median Salary", "Mid-Career 10th Percentile Salary", "Mid-Career 25th Percentile Salary", "Mid-Career 75th Percentile Salary", "Mid-Career 90th Percentile Salary"]
combined = college.merge(region, how='outer', on=cols)
combined.head()

,School Name,School Type,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary,Region
0,Massachusetts Institute of Technology (MIT),Engineering,"$72,200.00","$126,000.00","$76,800.00","$99,200.00","$168,000.00","$220,000.00",Northeastern
1,California Institute of Technology (CIT),Engineering,"$75,500.00","$123,000.00",NaN,"$104,000.00","$161,000.00",NaN,California
2,Harvey Mudd College,Engineering,"$71,800.00","$122,000.00",NaN,"$96,000.00","$180,000.00",NaN,California
3,"Polytechnic University of New York, Brooklyn",Engineering,"$62,400.00","$114,000.00","$66,800.00","$94,300.00","$143,000.00","$190,000.00",Northeastern
4,Cooper Union,Engineering,"$62,200.00","$114,000.00",NaN,"$80,200.00","$142,000.00",NaN,Northeastern


In [30]:
combined.isnull().sum()

School Name                           0
School Type                          80
Starting Median Salary                0
Mid-Career Median Salary              0
Mid-Career 10th Percentile Salary    49
Mid-Career 25th Percentile Salary     0
Mid-Career 75th Percentile Salary     0
Mid-Career 90th Percentile Salary    49
Region                                9
dtype: int64

In [31]:
combined.shape

(349, 9)

In [32]:
missing = combined[combined["Mid-Career 10th Percentile Salary"].isnull()]
missing.head()

,School Name,School Type,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary,Region
1,California Institute of Technology (CIT),Engineering,"$75,500.00","$123,000.00",NaN,"$104,000.00","$161,000.00",NaN,California
2,Harvey Mudd College,Engineering,"$71,800.00","$122,000.00",NaN,"$96,000.00","$180,000.00",NaN,California
4,Cooper Union,Engineering,"$62,200.00","$114,000.00",NaN,"$80,200.00","$142,000.00",NaN,Northeastern
15,New Mexico Institute of Mining and Technology ...,Engineering,"$51,000.00","$93,400.00",NaN,"$67,400.00","$123,000.00",NaN,Western
43,Randolph-Macon College,Party,"$42,600.00","$83,600.00",NaN,"$54,100.00","$123,000.00",NaN,Southern


In [33]:
# If 25th percentile == x, and 75th percentile == y, and we assume that it's roughly normal, then we can find a rough mean,
# and use z-score to calculate the 10th and 90th percentile to fill in our data.
missing.where(missing["Mid-Career 10th Percentile Salary"].isnull()).index \
== missing.where(missing["Mid-Career 90th Percentile Salary"].isnull()).index

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [34]:
s# Any guesses for the median starting salaries?
np.mean(missing["Mid-Career 25th Percentile Salary"])

TypeError: Could not convert $104,000.00$96,000.00$80,200.00$67,400.00$54,100.00$54,100.00$84,900.00$74,600.00$65,600.00$54,800.00$82,800.00$67,200.00$70,500.00$69,400.00$76,400.00$63,300.00$60,700.00$60,600.00$69,100.00$57,100.00$55,800.00$63,500.00$66,300.00$51,900.00$58,600.00$62,100.00$46,400.00$43,000.00$67,400.00$49,300.00$64,800.00$67,200.00$54,100.00$65,100.00$42,800.00$56,700.00$55,000.00$44,600.00$50,500.00$44,100.00$39,700.00$31,800.00$32,200.00$46,100.00$45,400.00$43,900.00$37,100.00$44,500.00$57,100.00 to numeric

### It errors! Why?
If you look closely, the data is actually strings that start with a dollar sign and includes commas. Without thinking too much about it, we don't realize that the commas and dollar signs are even there -- it's just intuitive for us, but the computer sees it. Let's fix it.

In [ ]:
missing["Mid-Career 25th Percentile Salary"] = missing["Mid-Career 25th Percentile Salary"].copy().str.replace(r"\$|,", "").astype(float);
missing["Mid-Career 75th Percentile Salary"] = missing["Mid-Career 75th Percentile Salary"].copy().str.replace(r"\$|,", "").astype(float);

In [ ]:
mus = (missing["Mid-Career 25th Percentile Salary"] + missing["Mid-Career 75th Percentile Salary"]) / 2
mus.head()

In [ ]:
with_mus = missing.copy()
with_mus["Mid-Career 50th Percentile Salary"] = mus
with_mus.head()

In [ ]:
# If 25th percentile == x, and 75th percentile == y, and we assume that it's roughly normal, then we can find a rough mean,
# and use z-score to calculate the 10th and 90th percentile to fill in our data.
z_10 = -1.282
z_25 = -0.675
z_75 = 0.675
z_90 = 1.282

# Z = (X - mu) / sigma; ex: for Caltech, z_25 = (123,000 - 132500) / sigma; sigma = (123,000 - 132,500) / -0.675
sigmas_from_25 = (with_mus["Mid-Career 25th Percentile Salary"] -  with_mus["Mid-Career 50th Percentile Salary"]) / -0.675
sigmas_from_25.head()

In [ ]:
# Check that we get the same sigmas from
sigmas_from_75 = (with_mus["Mid-Career 25th Percentile Salary"] -  with_mus["Mid-Career 50th Percentile Salary"]) / -0.675
sigmas_from_75.head()

In [ ]:
with_mus_sds = with_mus.copy()
with_mus_sds["Mid-Career SD"] = sigmas_from_25
with_mus_sds.head()

In [ ]:
# Z_10 = (X_10 - mu) / sigma; X_10 = Z_10 * sigma + mu
filled = with_mus_sds.copy()
filled["Mid-Career 10th Percentile Salary"] = z_10 * filled["Mid-Career SD"] + filled["Mid-Career 50th Percentile Salary"]

# Z_90 = (X_90 - mu) / sigma; X_90 = Z_90 * sigma + mu
filled["Mid-Career 90th Percentile Salary"] = z_90 * filled["Mid-Career SD"] + filled["Mid-Career 50th Percentile Salary"]

In [ ]:
filled.head()

In [ ]:
final = filled.merge(combined, how="left", on=cols + ["School Type", "Region"]).head()
final.head()

In [ ]:
final.isnull().sum()

Good job getting through this document!
There's plenty of other ways to clean data, but here's just a couple to start off :)
Ask your mentors questions if you are confused!
-Vincent Lao